In [2]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 9
batch_size = 16
data_dir = "/kaggle/input/AI-OF-GOD-4/aog_data/test/images"

efficientnet_paths = [
    "/kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold1.pth",
    "/kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold2.pth",
    "/kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold3.pth",
]

swin_paths = [
    "/kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold1_epoch10.pth",
    "/kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold2_epoch9.pth",
    "/kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold3_epoch10.pth",
]

test_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.CenterCrop(384),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class TestImageDataset(torch.utils.data.Dataset):
    def __init__(self, folder, transform):
        self.paths = sorted([
            os.path.join(folder, fname)
            for fname in os.listdir(folder)
            if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, os.path.basename(img_path)

test_dataset = TestImageDataset(data_dir, test_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

def load_efficientnet_model(path):
    model = models.efficientnet_v2_l(weights=None)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)
    state_dict = torch.load(path, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model.to(device)
    model.eval()
    return model

def load_swin_model(path):
    model = models.swin_v2_s(weights=None)
    in_features = model.head.in_features
    model.head = nn.Linear(in_features, num_classes)
    state_dict = torch.load(path, map_location=device)
    if any(k.startswith("module.") for k in state_dict.keys()):
        state_dict = {k.replace("module.", "", 1): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict, strict=False)
    model.to(device)
    model.eval()
    return model

def get_ensemble_predictions(model_paths, loader, model_loader_fn):
    all_probs = []
    for path in model_paths:
        print(f"Loading model from {path}")
        model = model_loader_fn(path)
        probs = []
        with torch.no_grad():
            for inputs, _ in tqdm(loader, leave=False):
                inputs = inputs.to(device)
                outputs = model(inputs)
                preds = torch.softmax(outputs, dim=1)
                probs.append(preds.cpu().numpy())
        probs = np.concatenate(probs, axis=0)
        all_probs.append(probs)
    ensemble_probs = np.mean(all_probs, axis=0)
    return ensemble_probs

print("\n🔹 Generating EfficientNetV2-L ensemble predictions...")
effnet_preds = get_ensemble_predictions(efficientnet_paths, test_loader, load_efficientnet_model)

print("\n🔹 Generating SwinV2-S ensemble predictions...")
swin_preds = get_ensemble_predictions(swin_paths, test_loader, load_swin_model)

final_probs = (effnet_preds + swin_preds) / 2
final_labels = np.argmax(final_probs, axis=1)

output_file = "ensemble_predictions.csv"
with open(output_file, "w") as f:
    f.write("filename,label\n")
    for (_, filename), label in zip(test_dataset, final_labels):
        f.write(f"{filename},{label}\n")

print(f"\n✅ Ensemble inference complete! Saved results to: {output_file}")


🔹 Generating EfficientNetV2-L ensemble predictions...
Loading model from /kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold1.pth


Loading model from /kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold2.pth


Loading model from /kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold3.pth



🔹 Generating SwinV2-S ensemble predictions...
Loading model from /kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold1_epoch10.pth


Loading model from /kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold2_epoch9.pth


Loading model from /kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold3_epoch10.pth



✅ Ensemble inference complete! Saved results to: ensemble_predictions.csv


In [3]:
import os
import ast
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 9
batch_size = 16
data_dir = "/kaggle/input/AI-OF-GOD-4/aog_data/test/images"
maxvit_csv = "/kaggle/input/aiofgod-vitmax-ensemble-preds/maxvit_5fold_probs.csv"

swin_paths = [
    "/kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold1_epoch10.pth",
    "/kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold2_epoch9.pth",
    "/kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold3_epoch10.pth",
]

test_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.CenterCrop(384),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class TestImageDataset(torch.utils.data.Dataset):
    def __init__(self, folder, transform):
        self.paths = sorted([
            os.path.join(folder, fname)
            for fname in os.listdir(folder)
            if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, os.path.basename(img_path)

test_dataset = TestImageDataset(data_dir, test_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

def load_swin_model(path):
    model = models.swin_v2_s(weights=None)
    in_features = model.head.in_features
    model.head = nn.Linear(in_features, num_classes)
    state_dict = torch.load(path, map_location=device)
    if any(k.startswith("module.") for k in state_dict.keys()):
        state_dict = {k.replace("module.", "", 1): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict, strict=False)
    model.to(device)
    model.eval()
    return model

def get_ensemble_predictions(model_paths, loader, model_loader_fn):
    all_probs = []
    for path in model_paths:
        print(f"Loading model from {path}")
        model = model_loader_fn(path)
        probs = []
        with torch.no_grad():
            for inputs, _ in tqdm(loader, leave=False):
                inputs = inputs.to(device)
                outputs = model(inputs)
                preds = torch.softmax(outputs, dim=1)
                probs.append(preds.cpu().numpy())
        probs = np.concatenate(probs, axis=0)
        all_probs.append(probs)
    return np.mean(all_probs, axis=0)

print("\n🔹 Loading MaxViT 5-fold predictions...")
df = pd.read_csv(maxvit_csv)
for col in [c for c in df.columns if "fold_" in c]:
    df[col] = df[col].apply(lambda x: np.array(ast.literal_eval(x), dtype=np.float32))

prob_arrays = np.stack([np.vstack(df[c].to_numpy()) for c in df.columns if "fold_" in c], axis=0)
maxvit_ensemble_probs = prob_arrays.mean(axis=0)
print(f"✅ Averaged {prob_arrays.shape[0]} MaxViT folds → shape {maxvit_ensemble_probs.shape}")

print("\n🔹 Generating SwinV2-S ensemble predictions...")
swin_preds = get_ensemble_predictions(swin_paths, test_loader, load_swin_model)

final_probs = (maxvit_ensemble_probs + swin_preds) / 2
final_labels = np.argmax(final_probs, axis=1)

output_file = "final_ensemble_predictions.csv"
with open(output_file, "w") as f:
    f.write("filename,label\n")
    for (_, filename), label in zip(test_dataset, final_labels):
        f.write(f"{filename},{label}\n")

print(f"\n✅ Final ensemble complete! Saved results to: {output_file}")


🔹 Loading MaxViT 5-fold predictions...
✅ Averaged 5 MaxViT folds → shape (5253, 9)

🔹 Generating SwinV2-S ensemble predictions...
Loading model from /kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold1_epoch10.pth


Loading model from /kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold2_epoch9.pth


Loading model from /kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold3_epoch10.pth



✅ Final ensemble complete! Saved results to: final_ensemble_predictions.csv


In [4]:
import os
import ast
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 9
batch_size = 16
data_dir = "/kaggle/input/AI-OF-GOD-4/aog_data/test/images"

efficientnet_paths = [
    "/kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold1.pth",
    "/kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold2.pth",
    "/kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold3.pth",
]

swin_paths = [
    "/kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold1_epoch10.pth",
    "/kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold2_epoch9.pth",
    "/kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold3_epoch10.pth",
]

maxvit_csv = "/kaggle/input/aiofgod-vitmax-ensemble-preds/maxvit_5fold_probs.csv"

test_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.CenterCrop(384),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class TestImageDataset(torch.utils.data.Dataset):
    def __init__(self, folder, transform):
        self.paths = sorted([
            os.path.join(folder, fname)
            for fname in os.listdir(folder)
            if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, os.path.basename(img_path)

test_dataset = TestImageDataset(data_dir, test_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

def load_efficientnet_model(path):
    model = models.efficientnet_v2_l(weights=None)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)
    state_dict = torch.load(path, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model.to(device)
    model.eval()
    return model

def load_swin_model(path):
    model = models.swin_v2_s(weights=None)
    in_features = model.head.in_features
    model.head = nn.Linear(in_features, num_classes)
    state_dict = torch.load(path, map_location=device)
    if any(k.startswith("module.") for k in state_dict.keys()):
        state_dict = {k.replace("module.", "", 1): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict, strict=False)
    model.to(device)
    model.eval()
    return model

def get_ensemble_predictions(model_paths, loader, model_loader_fn):
    all_probs = []
    for path in model_paths:
        print(f"Loading model from {path}")
        model = model_loader_fn(path)
        probs = []
        with torch.no_grad():
            for inputs, _ in tqdm(loader, leave=False):
                inputs = inputs.to(device)
                outputs = model(inputs)
                probs.append(torch.softmax(outputs, dim=1).cpu().numpy())
        all_probs.append(np.concatenate(probs, axis=0))
    return np.mean(all_probs, axis=0)

print("\n🔹 Loading MaxViT 5-fold predictions...")
df = pd.read_csv(maxvit_csv)
for col in [c for c in df.columns if "fold_" in c]:
    df[col] = df[col].apply(lambda x: np.array(ast.literal_eval(x), dtype=np.float32))

prob_arrays = np.stack([np.vstack(df[c].to_numpy()) for c in df.columns if "fold_" in c], axis=0)
maxvit_ensemble_probs = prob_arrays.mean(axis=0)
print(f"✅ MaxViT ensemble shape: {maxvit_ensemble_probs.shape}")

print("\n🔹 Generating EfficientNetV2-L ensemble predictions...")
effnet_preds = get_ensemble_predictions(efficientnet_paths, test_loader, load_efficientnet_model)

print("\n🔹 Generating SwinV2-S ensemble predictions...")
swin_preds = get_ensemble_predictions(swin_paths, test_loader, load_swin_model)

final_probs = (maxvit_ensemble_probs + effnet_preds + swin_preds) / 3
final_labels = np.argmax(final_probs, axis=1)

output_file = "final_ensemble_predictions.csv"
with open(output_file, "w") as f:
    f.write("filename,label\n")
    for (_, filename), label in zip(test_dataset, final_labels):
        f.write(f"{filename},{label}\n")

print(f"\n✅ Final ensemble (MaxViT + EfficientNet + Swin) complete! Saved results to: {output_file}")


🔹 Loading MaxViT 5-fold predictions...
✅ MaxViT ensemble shape: (5253, 9)

🔹 Generating EfficientNetV2-L ensemble predictions...
Loading model from /kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold1.pth


Loading model from /kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold2.pth


Loading model from /kaggle/input/efficientnet-v2_l-ensemble/pytorch/default/1/efficientnetv2l_best_fold3.pth



🔹 Generating SwinV2-S ensemble predictions...
Loading model from /kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold1_epoch10.pth


Loading model from /kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold2_epoch9.pth


Loading model from /kaggle/input/swin-v2_s/pytorch/default/1/best_model_fold3_epoch10.pth



✅ Final ensemble (MaxViT + EfficientNet + Swin) complete! Saved results to: final_ensemble_predictions.csv
